# Libraries

In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
import sympy as sym
from scipy.optimize import minimize, fsolve
import math
from matplotlib.patches import Arc
from bisect import bisect_left
import pandas as pd
from numpy import exp
import matplotlib as mpl
import os

from matplotlib.ticker import MultipleLocator

mpl.rc('font', family = 'DejaVu Serif')

# Important Functions

In [2]:
r_aux = sym.Symbol('r')

In [3]:
def RungeKutta4(f, a, L, u, N):
  h = (L - a) / N
  result = []
  grid = []
  result.append(u)
  grid.append(a)
  x = a
  for i in range(N):
    if (i+1) % (N/4) == 0:
      print('Complete: '+str(int((i+1)/N * 100))+'%')
    try:
      k1 = f(x, u)
      k2 = f(x + h / 2, u + h * k1 / 2)
      k3 = f(x + h / 2, u + h * k2 / 2)
      k4 = f(x + h, u + h * k3)
      u = u + h * (k1 + 2 * k2 + 2 * k3 + k4)/6
      x = x + h
    except:
      print("Point",x)
    result.append(u)
    grid.append(x)
  return np.array(grid), np.array(result)

def RungeKutta5(f, a, L, u, N):
  h = (L - a) / N
  result = []
  grid = []
  result.append(u)
  grid.append(a)
  x = a
  for i in range(N):
    if (i+1) % (N/4) == 0:
      print('Complete: '+str(int((i+1)/N * 100))+'%')
    k1 = f(x, u)
    k2 = f(x + h / 4, u + h * k1 / 4)
    k3 = f(x + h / 4, u + h * k1 / 8 + h * k2 / 8)
    k4 = f(x + h / 2, u - h * k2 / 2 + h * k3)
    k5 = f(x + 3 * h / 4, u + 3 * h * k1 / 16 + 9 * h * k4 / 16)
    k6 = f(x + h, u - 3 * h * k1 / 7 + 2 * h * k2 / 7 + 12 * h * k3 / 7 - 12 * h * k4 / 7 + 8 * h * k5 / 7)
    u = u + (h/90) * (7 * k1 + 32 * k3 + 12 * k4 + 32 * k5 + 7 * k6)
    x = x + h
    result.append(u)
    grid.append(x)
  return np.array(grid), np.array(result)

# Conditions

In [4]:
M = 1 # BH mass
gamma = 1.4

N = 2000 # nº of points

cond = np.array([#alpha,ell,Q,l multipole
                [0.1,0.2,0.1,0],
                [0.1,0.2,0.4,0],
                [0.1,0.2,1.0,0],

                [0.5,1.0,0.1,0],
                [0.5,1.0,0.4,0],
                [0.5,1.0,1.0,0],

                [0.1,0.2,1.0,0],
                [0.2,0.4,1.0,0],
                [0.4,0.8,1.0,0],
                [0.6,1.2,1.0,0],
                [0.8,1.6,1.0,0],


                [0.1,0.2,0.1,1],
                [0.1,0.2,0.4,1],
                [0.1,0.2,1.0,1],

                [0.5,1.0,0.1,1],
                [0.5,1.0,0.4,1],
                [0.5,1.0,1.0,1],

                [0.1,0.2,1.0,1],
                [0.2,0.4,1.0,1],
                [0.4,0.8,1.0,1],
                [0.6,1.2,1.0,1],
                [0.8,1.6,1.0,1],
                ]).T

print(len(cond[0]))
alpha = cond[0]
ell = cond[1]
Q = cond[2]
l = cond[3]

22


# Main Function

In [5]:
def QNmode_Aerodynamics(alpha, ell, Q,l_mult):
  print("******************* alpha =",alpha,"| ell =", ell,"| Q =", Q)
  Q = Q
  alpha = alpha
  l_met = ell
  l = l_mult

  offset = 1.005 # 0.5% to remove singularity

  fl_salva = True

  title_arq = r"alpha = {alpha} - Q = {Q} - ell = {l_met} - l = {l}".format(alpha=alpha,Q=Q,l_met=l_met,l=l)

  def B(r):
    return sym.simplify(1 - 2*M/r + Q**2/r**2 - alpha/r * (M - alpha*l_met/2)  * sym.exp(-r/(M-alpha*l_met/2)))
  Bb = B(r_aux)
  def A(r):
    return sym.simplify(1/B(r))
  Aa = A(r_aux)

  def B_val(r):
    return eval('{}'.format(Bb))
  def A_val(r):
    return eval('{}'.format(Aa))

  solve = fsolve(B_val, [.5, 2.5])
  r_HE = np.max(solve)
  print("r_he =",r_HE)
  x_i = r_HE * offset
  x_f = 10*x_i
  print("Title:",title_arq)
  print('x_i =',x_i)
  print('x_f =',x_f)
  print('step =',(x_f - x_i) / N)

  f_aux = sym.simplify((B(r_aux)/A(r_aux)))

  F_aux = sym.simplify((B(r_aux)/A(r_aux))**(1/2))
  def F(r):
    return eval('{}'.format((F_aux)))

  veff = sym.simplify(B(r_aux)*l*(l+1)/r_aux**2 + (1/(2*r_aux))*sym.diff(f_aux, r_aux))
  def V(r):
    return eval('{}'.format((veff)))

  V_dot_aux = sym.simplify(sym.diff(veff, r_aux))
  def V_dot(r):
    return eval('{}'.format((V_dot_aux)))

  F_dot_aux = sym.simplify(sym.diff(F_aux, r_aux))
  def F_dot(r):
    return eval('{}'.format((F_dot_aux)))

  A_dot_aux = sym.diff(A(r_aux),r_aux)
  def A_dot(r):
    return eval('{}'.format(A_dot_aux))

  B_dot_aux = sym.diff(B(r_aux),r_aux)
  def B_dot(r):
    return eval('{}'.format(B_dot_aux))

  A_dot_dot_aux = sym.diff(sym.diff(A(r_aux),r_aux),r_aux)
  def A_dot_dot(r):
    return eval('{}'.format(A_dot_dot_aux))

  B_dot_dot_aux = sym.diff(sym.diff(B(r_aux),r_aux),r_aux)
  def B_dot_dot(r):
    return eval('{}'.format(B_dot_dot_aux))

  def calculate_K(r):
    term1 = (B_dot_dot_aux / (A(r) * B(r))) ** 2
    term2 = (B_dot_aux ** 2 / (2 * A(r) * B(r) ** 2)) ** 2
    term3 = (A_dot_aux * B_dot_aux / (2 * A(r) ** 2 * B(r))) ** 2
    term4 = -(B_dot_dot_aux * B_dot_aux ** 2 / (A(r) ** 2 * B(r) ** 2)) * (A_dot_aux / A(r) + B_dot_aux / B(r))
    term5 = A_dot_aux * B_dot_aux ** 3 / (2 * A(r) ** 3 * B(r) ** 3)
    term6 = 2 / (r ** 2 * A(r) ** 2) * ((A_dot_aux ** 2) / A(r) ** 2 + (B_dot_aux ** 2) / B(r) ** 2)
    term7 = 4 / r ** 4 * (1 - 1 / A(r)) ** 2
    K = sym.simplify(term1 + term2 + term3 + term4 + term5 + term6 + term7)
    return K

  first_term_aux = sym.simplify(veff/F_aux**2)
  def first_term(r):
    return eval('{}'.format(first_term_aux))

  second_term_aux = sym.simplify((-F_dot_aux)/F_aux)
  def second_term(r):
    return eval('{}'.format(second_term_aux))

  def f(t, u): # lambda's EDO
    f = np.zeros(2)
    f[0] = u[1]
    f[1] = first_term(t)*u[0] + second_term(t)*u[1]
    return f

  g_HE = (gamma + 1)/(2 * np.sqrt(2) * np.sqrt(gamma - 1))


  aprox_lamb = np.sqrt(g_HE) * sym.simplify((
      1
      + 0 * (r_aux - r_HE)
      + veff/(2*F_aux**2) * (r_aux - r_HE)**2
      - (1/6)*(veff/(F_aux**2))*(F_dot_aux/F_aux) * (r_aux - r_HE)**3
      ))
  aprox_lamb_dot = np.sqrt(g_HE) * (
      veff/F_aux**2 * (r_aux - r_HE)
      - (1/2) * (veff/F_aux**2)*(F_dot_aux/F_aux) * (r_aux - r_HE)**2
      )

  def lamb_frob(r):
    return eval('{}'.format(aprox_lamb))

  def lamb_dot_frob(r):
    return eval('{}'.format(aprox_lamb_dot))

  def u0(s):
    return np.array([lamb_frob(x_i),lamb_dot_frob(x_i)])
  u2 = 0

  u = u0(0)
  print("** Integration of g")
  grid, result = RungeKutta4(f, x_i, x_f, u, N)
  result = result**2

  nome_graf = 'Function g(r) - '+title_arq+'.xlsx'
  dados = np.array([grid,result[:,0],result[:,1]])
  nomes = ["r",'g','g_dot']
  if fl_salva:
    pd.DataFrame(dados,nomes).T.to_excel(nome_graf)

  for i in range(len(result[:,0])):
    if result[:,0][i] <= 1:
      print("Erro em x =",grid[i]," g =",result[:,0][i])

  gf = interpolate.interp1d(grid, result[:, 0], fill_value='extrapolate')
  gf_dot = interpolate.interp1d(grid, result[:, 1], fill_value='extrapolate')

  def r_to_x(r,y):
    return (gamma+1-4*gf(r)**2 *(1-(1-gf(r)**(-2))**(1/2)))\
            /(F(r)*(gamma - 1)*(2*gf(r)**2*(1-(1-gf(r)**(-2))**(1/2)))**(1/2))


  print("** Integration of r(x)")
  grid, result = RungeKutta4(r_to_x, x_i, x_f, u2, N)
  print("\nCheck:",min(result)>=0)

  nome_graf = 'Function r(x) - '+title_arq+'.xlsx'
  dados = np.array([result,grid])
  nomes = ['x','r']
  if fl_salva:
    pd.DataFrame(dados,nomes).T.to_excel(nome_graf)

  def At(r):
    return ((np.sqrt(2)*\
              (2*gf(r)**2*(1-np.sqrt(1-gf(r)**(-2))))**(1/(gamma-1)))\
            /(1-np.sqrt(1-gf(r)**(-2)))**(1/2))

  equalizacao = min(At(grid))
  A_aux = At(grid)/equalizacao
  A = A_aux.astype(float)
  A_nozzle = np.sort(np.append(-A,A))
  result = result.astype(float)

  nome_graf = 'Nozzle Area - '+title_arq+'.xlsx'
  dados = np.array([-result, np.sqrt(A)])
  nomes = ['x','A/A*']
  if fl_salva:
    pd.DataFrame(dados,nomes).T.to_excel(nome_graf)

  epsilon = 1e-30
  chute_M = 1

  def M0(M):
    return (2/(gamma+1) * (1+(gamma-1)/2 * M**2))**((gamma+1)/(2*(gamma-1))) /A
  def M1(M):
    return np.sqrt((2/(gamma-1))*(((gamma+1)/2)*(A*M)**(2*(gamma-1)/(gamma+1)) - 1))

  M0_calc = M0(chute_M)
  M1_calc = M1(chute_M)

  ite = 100000
  for i in range(1,ite):
    M0_calc_aux = M0(M0_calc)
    M1_calc_aux = M1(M1_calc)
    if np.max(np.abs(M1_calc-M1_calc_aux))<epsilon and np.max(np.abs(M0_calc-M0_calc_aux))<epsilon:
      print(i)
      break
    M0_calc = M0_calc_aux
    M1_calc = M1_calc_aux

  M_nozzle = np.sort(np.append(M0_calc_aux,M1_calc_aux))
  x_nozzle = np.sort(np.append(-result,result))

  nome_graf = 'Mach Number - '+title_arq+'.xlsx'
  dados = np.array([x_nozzle,M_nozzle])
  nomes = ['x','M']
  if fl_salva:
    pd.DataFrame(dados,nomes).T.to_excel(nome_graf)

  def func_T(M):
    return (1+((gamma-1)/2) * M**2)**(-1)

  def func_P(M):
    return (1+((gamma-1)/2) * M**2)**(-gamma/(gamma-1))

  def func_rho(M):
    return (1+((gamma-1)/2) * M**2)**(-1/(gamma-1))

  def CF_Vacuo(M,A):
    return np.sqrt((2*gamma**2/(gamma-1)) * (2/(gamma+1))**((gamma+1)/(gamma-1)) * (1 - func_P(M)**((gamma-1)/gamma) )) + A*func_P(M)

  def c_estrela(M):
    return np.sqrt(1/func_T(M)) * (1/gamma) * ((gamma+1)/2)**(1/(gamma-1))

  def empuxo_mdot(M,A):
    return c_estrela(M)*CF_Vacuo(M,A)

  CF_max_teorico = np.sqrt((2*gamma**2/(gamma-1)) * (2/(gamma+1))**((gamma+1)/(gamma-1)) )

  nome_graf = 'Pressure - '+title_arq+'.xlsx'
  dados = np.array([x_nozzle,func_P(M_nozzle)])
  nomes = ['x','P']
  if fl_salva:
    pd.DataFrame(dados,nomes).T.to_excel(nome_graf)

  nome_graf = 'Temperature - '+title_arq+'.xlsx'
  dados = np.array([x_nozzle,func_T(M_nozzle)])
  nomes = ['x','T']
  if fl_salva:
    pd.DataFrame(dados,nomes).T.to_excel(nome_graf)

  nome_graf = 'Density - '+title_arq+'.xlsx'
  dados = np.array([x_nozzle,func_rho(M_nozzle)])
  nomes = ['x','rho']
  if fl_salva:
    pd.DataFrame(dados,nomes).T.to_excel(nome_graf)

  nome_graf = 'Thrust Coefficient - '+title_arq+'.xlsx'
  dados = np.array([x_nozzle,CF_Vacuo(M_nozzle,A_nozzle)])
  nomes = ['x','C_F']
  if fl_salva:
    pd.DataFrame(dados,nomes).T.to_excel(nome_graf)

  nome_graf = 'cstar - '+title_arq+'.xlsx'
  dados = np.array([x_nozzle,c_estrela(M_nozzle)])
  nomes = ['x','c*']
  if fl_salva:
    pd.DataFrame(dados,nomes).T.to_excel(nome_graf)

  nome_graf = 'Thrust - '+title_arq+'.xlsx'
  dados = np.array([x_nozzle,empuxo_mdot(M_nozzle,A_nozzle)])
  nomes = ['x','F']
  if fl_salva:
    pd.DataFrame(dados,nomes).T.to_excel(nome_graf)

# Loop

In [6]:
for i in range(len(alpha)):
  print("---------> Now:",i+1,'of',len(alpha))
  QNmode_Aerodynamics(alpha[i], ell[i], Q[i], l[i])